In [1]:
%matplotlib inline
import torch
import kornia as K
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
# data: two patch sets from HPatches https://github.com/hpatches/hpatches-dataset
dir_fname = 'data/hpatches_selected/'
pairs = [('v_graffiti_e1', 'v_graffiti_h4'),
         ('i_porta_e1', 'i_porta_h4')]

In [2]:
from kornia.feature import SIFTDescriptor

def benchmark_matching_score(P1, P2, descriptor):
    #perm1 = np.random.permutation(P1.size(0))
    #perm1_inv = np.argsort(perm1)
    #P1 = P1[perm1]
    with torch.no_grad():
        desc1 = descriptor(P1)
        #desc1 = desc1[perm1_inv]
        desc2 = descriptor(P2)
        dm = torch.cdist(desc1,desc2)
        ind_in_2 = dm.min(dim=1)[1]
        correct = ind_in_2.view(-1) == torch.arange(P1.size(0))
        acc = correct.float().mean().item() * 100
    return acc
    

In [3]:
descriptor = K.feature.SIFTDescriptor(32, rootsift=True)
accs = []
for f1, f2 in pairs:
    fn1 = os.path.join(dir_fname, f1 + '.png')
    fn2 = os.path.join(dir_fname, f2 + '.png')
    print (fn1, fn2)
    P1 = K.image_to_tensor(np.array(Image.open(fn1).convert('L'))).float()/255.
    P2 = K.image_to_tensor(np.array(Image.open(fn2).convert('L'))).float()/255.
    P1 = P1.reshape(-1,1,65,65)
    P2 = P2.reshape(-1,1,65,65)
    acc = benchmark_matching_score(K.resize(P1,(32,32)), K.resize(P2,(32,32)), descriptor)
    print (f' matching score = {acc:.1f} %')
    accs.append(acc)
print (f'overall matching score = {torch.tensor(accs).mean().item():.1f} %')


data/hpatches_selected/v_graffiti_e1.png data/hpatches_selected/v_graffiti_h4.png
 matching score = 33.9 %
data/hpatches_selected/i_porta_e1.png data/hpatches_selected/i_porta_h4.png
 matching score = 22.5 %
overall matching score = 28.2 %


In [4]:
descriptor = K.feature.HardNet(True)
accs = []
for f1, f2 in pairs:
    fn1 = os.path.join(dir_fname, f1 + '.png')
    fn2 = os.path.join(dir_fname, f2 + '.png')
    print (fn1, fn2)
    P1 = K.image_to_tensor(np.array(Image.open(fn1).convert('L'))).float()/255.
    P2 = K.image_to_tensor(np.array(Image.open(fn2).convert('L'))).float()/255.
    P1 = P1.reshape(-1,1,65,65)
    P2 = P2.reshape(-1,1,65,65)
    acc = benchmark_matching_score(K.resize(P1,(32,32)), K.resize(P2,(32,32)), descriptor)
    print (f' matching score = {acc:.1f} %')
    accs.append(acc)
print (f'overall matching score = {torch.tensor(accs).mean().item():.1f} %')


data/hpatches_selected/v_graffiti_e1.png data/hpatches_selected/v_graffiti_h4.png
 matching score = 57.7 %
data/hpatches_selected/i_porta_e1.png data/hpatches_selected/i_porta_h4.png
 matching score = 46.5 %
overall matching score = 52.1 %
